# Function calling with OpenAPI Spec

In [2]:
!pip install -q jsonref
!pip install -q openai


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import uuid
import os
import json
from typing import Optional
from pydantic import BaseModel, Extra
import requests
import jsonref
import re
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [42]:
def parse_functions(openapi_spec):
    
    name_map = {}
    
    # List to hold the details of the generated functions
    functions = []

    # Iterate through the paths and methods specified in the OpenAPI spec
    for path, methods in openapi_spec["paths"].items():
        for method, spec_with_ref in methods.items():
            spec = jsonref.replace_refs(spec_with_ref)
            function_name = spec.get("operationId")
            name_map[function_name] = {"method": method, "path": path}
            
            # Gather the description, request body and parameters from the spec
            desc = spec.get("description") or spec.get("summary", "")
            req_body = spec.get("requestBody", {}).get("content", {}).get("application/json", {}).get("schema")
            params = spec.get("parameters", [])
            
            schema = {"type": "object", "properties": {}}
            if req_body:
                schema["properties"]["requestBody"] = req_body
            
            if params:
                param_properties = {param["name"]: param["schema"] for param in params}
                schema["properties"]["parameters"] = {"type": "object", "properties": param_properties}
            
            functions.append({"name": function_name, "description": desc, "parameters": schema})

    return functions, name_map

In [43]:
url = "https://gist.githubusercontent.com/shyamal-anadkat/d44674a87778796222bdb8fa9158ad47/raw/030d173d53c55d806a93976705cf1c5f9e9c5240/frogeapi.json"
openapi_spec = jsonref.loads(requests.get(url).content)

functions, names = parse_functions(openapi_spec)

In [61]:
FROGE_PROMPT = """
You are a froge. Respond to the following prompt using a function_call:

Get all the froges. Then create a new 1-year old froge named dalle3. Then look for a froge named davinci. Then delete the davinci froge.
"""

messages = [
    {"role": "user", "content": FROGE_PROMPT}
]

MAX_CHAINED_CALLS = 5

def get_openai_response(functions, messages):
    return openai.ChatCompletion.create(
        model='gpt-3.5-turbo-16k-0613',
        functions=functions,
        function_call="auto",
        temperature=0,
        messages=messages
    )

def process_chained_calls(functions, messages):
    for stack in range(MAX_CHAINED_CALLS):
        response = get_openai_response(functions, messages)
        message = response["choices"][0]["message"]
        print(message)
        
        if message.get("function_call"):
            messages.append(message)
        else:
            break
    else:
        print(f"Reached max chained function calls: {MAX_CHAINED_CALLS}")


process_chained_calls(functions, messages)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "listFroges",
    "arguments": "{}"
  }
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "createFroge",
    "arguments": "{\n  \"requestBody\": {\n    \"id\": \"dalle3\",\n    \"name\": \"dalle3\",\n    \"age\": 1\n  }\n}"
  }
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "getFrogeByName",
    "arguments": "{\n  \"parameters\": {\n    \"name\": \"davinci\"\n  }\n}"
  }
}
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "deleteFroge",
    "arguments": "{\n  \"parameters\": {\n    \"id\": \"davinci\"\n  }\n}"
  }
}
{
  "role": "assistant",
  "content": "Here are the steps to perform the requested actions:\n\n1. Get all the froges.\n2. Create a new 1-year old froge named dalle3.\n3. Look for a froge named davinci.\n4. Delete the davinci froge.\n\nPlease note that the response for each step will depend on the current state o

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=baca1bec-71a4-4a27-a9d1-210d39960b44' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>